<a href="https://colab.research.google.com/github/arunsechergy/TSAI-DeepNLP-END/blob/main/assignments/assignment14/Tokenize_Python_code_generate_predictions_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analysis:
1. Model is able to generate python code
2. Understands indendation and newline, colon
3. Inaccurate python code, need more understanding on the logic to be written
4. need better examples
5. need complete results

Next steps:
1. Increase number of epochs for learning (if needed, tweak learning rate)
2. Add Data Augmentation strategy
3. Any other loss function, needs to be checked 
4. Add Pretrained Python embedding layer



In [4]:
file = 'drive/MyDrive/datasets/english_python_data_tabs.txt'

In [5]:
mkdir data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!cp -r  'drive/MyDrive/datasets/english_python_data_tabs.txt' 'data/'

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
#from torchtext.data import Field, BucketIterator
from torchtext.legacy.data import Example, Field, BucketIterator, Dataset

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [73]:
import tokenize
import os
import re
import pandas as pd
from io import BytesIO, StringIO

In [11]:
DATA_DIR = 'data'
corpus_name = "english_python_data_tabs.txt"
# corpus_name = "english_python_data.txt"
corpus = os.path.join(DATA_DIR, corpus_name)

def readLines(file, n=None):
    with open(file, 'r') as datafile:
        lines = datafile.readlines()
    return lines
    # n = len(lines) if n is None else n
    # for line in lines[:n]:
    #     print(line)

input_file= readLines(corpus)

In [14]:
q_c = {}
prev_value = []
for id, line in enumerate(input_file):
  #if line.startswith('#'):
  #if (line.lower().startswith('# write'))  or (line.lower().startswith('#write')) or (line.lower().startswith('#python')) or (line.lower().startswith('# python')):
  if re.match(r'[#]\d*\s*(Write|write|python|Python)', line):  
    q_c[line] = id
    prev_line = line
    prev_value = []
  else:
    prev_value.append(line)
    q_c[prev_line] = prev_value

In [15]:
question_code_pairs = pd.DataFrame.from_dict(q_c.items())
question_code_pairs.columns = ['question', 'code']

In [16]:
question_code_pairs['code'] = question_code_pairs['code'].apply(lambda x: "".join(x))

In [17]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [18]:
spacy_en = spacy.load('en')

In [21]:
question_code_pairs['code']

0       num1 = 5465461\nnum2 = 8765468\nsum = num1 + n...
1       def add_two_numbers(num1, num2):\n\tsum = num1...
2       num1 = 123\nnum2 = 125\nnum3 = 148\nif (num1 >...
3       num1 = 100\nnum2 = 200\nnum3 = 300\nif (num1 <...
4        def merge_lists(l1, l2):\n\treturn l1 + l2\n\n\n
                              ...                        
2130      a = 60\nb = 13\n\nc = a | b\nprint("OR", c)\n\n
2131     a = 60\nb = 13\n\nc = a ^ b\nprint("XOR", c)\n\n
2132    a = 60\n\nc = ~a\nprint("Binary Ones Complemen...
2133        c = a << 2\nprint("Binary Left Shift", c)\n\n
2134         c = a >> 2\nprint("Binary Right Shift", c)\n
Name: code, Length: 2135, dtype: object

In [37]:
question_code_pairs['code'][1]

'def add_two_numbers(num1, num2):\n\tsum = num1 + num2\n\treturn sum\n\n\n'

In [39]:
# Test Tokenization
s=question_code_pairs['code'][1]

tokens = tokenize.tokenize(BytesIO(s.encode('utf-8')).readline)
print([token.string for token in tokens])

['utf-8', 'def', 'add_two_numbers', '(', 'num1', ',', 'num2', ')', ':', '\n', '\t', 'sum', '=', 'num1', '+', 'num2', '\n', 'return', 'sum', '\n', '\n', '\n', '', '']


In [54]:
def tokenize_en(text, max_length=200):
    """
    Tokenizes English text from a string into a list of strings
    """
    tokens = [tok.text for tok in spacy_en.tokenizer(text)]
    tokens = tokens[:max_input_length]
    return tokens

In [79]:
def tokenize_py(text, max_length=200):
  """
  Tokenizes Python code from a string into a list of strings
  """
  tokenized_code = []
  tokens = tokenize.tokenize(BytesIO(text.encode('utf-8')).readline)
  try:
    for token in tokens:
      tokenized_code.append(token.string)
    # tokens = [token.string for token in tokens]
  except Exception as e:
    pass
    # print(f"Error in tokenization {e}")

  tokens = tokenized_code[:max_input_length]
  return tokens

In [81]:
SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = tokenize_py, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True
            )

In [82]:
fields = [('src', SRC), ('trg', TRG)]

In [83]:
train_len = int(0.8*(question_code_pairs.shape[0]))
valid_len = int(0.9*(question_code_pairs.shape[0]))
test_len = int((question_code_pairs.shape[0]))

train_len, valid_len, test_len

(1708, 1921, 2135)

In [84]:
train_pair, valid_pair, test_pair = question_code_pairs[:train_len], question_code_pairs[train_len:valid_len], question_code_pairs[valid_len:test_len]

In [85]:
train_example = [Example.fromlist([train_pair.question[i], train_pair.code[i]], fields) for i in range(train_len)]
train_data = Dataset(train_example, fields)

In [86]:
valid_example = [Example.fromlist([valid_pair.question[i], valid_pair.code[i]], fields) for i in range(train_len, valid_len)]
valid_data = Dataset(valid_example, fields)

In [87]:
test_example = [Example.fromlist([test_pair.question[i], test_pair.code[i]], fields) for i in range(valid_len, test_len)]
test_data = Dataset(test_example, fields)

In [88]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [92]:
# checking :, \t and \n in the vocab
vars(TRG.vocab)['stoi'].get(':'), vars(TRG.vocab)['stoi'].get('\t'), vars(TRG.vocab)['stoi'].get('\n')

(9, 14, 4)

In [94]:
# check max sequence length of the train_data
len__ = []
for i in range(len(train_data)):
    len_ = len(vars(list(train_data)[i])['trg'])
    len__.append(len_)
max(len__)

100

In [95]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [96]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device,sort=False)

In [97]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]

        #print(f'{"encoder pos Embedding", self.pos_embedding(pos) }')
        #print(f'{"encoder pos Embedding shape", pos.shape }')
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [98]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [99]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [100]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [101]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
        #print(f"Shape of position: batch_size, trg_len {pos.shape} == {batch_size} {trg_len}")

        #print(f'{"postion", pos}')
        #print(f'{"token_embedding",self.tok_embedding(trg)}')
        #print(f'{"self scale", self.scale}')
        #print(f'{"self scale", self.pos_embedding(pos)}')
        #print(f'{"print target",(self.tok_embedding(trg) * self.scale)}')
        #print(f'{"print target",(self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos)}')


            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [102]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [103]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [104]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
MAX_SEQ_LENGTH = 200
enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device,
              MAX_SEQ_LENGTH)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device,
              MAX_SEQ_LENGTH)

In [105]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [106]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,325,806 trainable parameters


In [107]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [108]:
model.apply(initialize_weights);

In [109]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [110]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [111]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        #print(f'{"traget",trg}')
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
        #print(output)
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [112]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            #print(output)
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [113]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [114]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'python_code_generator-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 56s
	Train Loss: 5.978 | Train PPL: 394.534
	 Val. Loss: 4.799 |  Val. PPL: 121.430
Epoch: 02 | Time: 1m 52s
	Train Loss: 4.501 | Train PPL:  90.102
	 Val. Loss: 3.785 |  Val. PPL:  44.019
Epoch: 03 | Time: 1m 51s
	Train Loss: 3.710 | Train PPL:  40.835
	 Val. Loss: 3.150 |  Val. PPL:  23.345
Epoch: 04 | Time: 1m 51s
	Train Loss: 3.183 | Train PPL:  24.127
	 Val. Loss: 2.787 |  Val. PPL:  16.226
Epoch: 05 | Time: 1m 50s
	Train Loss: 2.856 | Train PPL:  17.391
	 Val. Loss: 2.588 |  Val. PPL:  13.309
Epoch: 06 | Time: 1m 57s
	Train Loss: 2.655 | Train PPL:  14.228
	 Val. Loss: 2.437 |  Val. PPL:  11.435
Epoch: 07 | Time: 1m 50s
	Train Loss: 2.490 | Train PPL:  12.067
	 Val. Loss: 2.334 |  Val. PPL:  10.319
Epoch: 08 | Time: 1m 49s
	Train Loss: 2.358 | Train PPL:  10.566
	 Val. Loss: 2.230 |  Val. PPL:   9.301
Epoch: 09 | Time: 1m 51s
	Train Loss: 2.232 | Train PPL:   9.315
	 Val. Loss: 2.162 |  Val. PPL:   8.688
Epoch: 10 | Time: 1m 49s
	Train Loss: 2.116 | Train PPL

In [137]:
model.load_state_dict(torch.load('python_code_generator-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 2.237 | Test PPL:   9.361 |


In [138]:
#Save the model in drive
!cp -r python_code_generator-model.pt 'drive/MyDrive/Models/'

In [121]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 200):
    # For Prediction
  # Set the model in evaluation mode to deactivate the DropOut modules
  # This is IMPORTANT to have reproducible results during evaluation!
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [116]:
# Check the predictions

In [128]:
example_idx = 11
src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']
print(src)
print(trg)

['#', 'write', 'a', 'python', 'function', 'to', 'reverse', 'a', 'user', 'provided', 'list', 'or', 'string']
['utf-8', '\n', 'def', 'reverse', '(', 'itr', ')', ':', '\n', '\t', 'return', 'itr', '[', ':', ':', '-', '1', ']', '', '', '']


In [129]:
translation, attention = translate_sentence(src, SRC, TRG, model, device, max_len=MAX_SEQ_LENGTH)
processed_prediction = ' '.join(translation[1:-1]) # spaced joined, takes from 1 because of utf-8 token and ignore the eos token with -1

print(f'predicted trg = {translation}')
print("translated prediction\n", ' '.join(translation[1:-1]))

predicted trg = ['utf-8', 'def', '<unk>', '(', 'string', ')', ':', '\n', '\t', 'return', 'list', '(', 'string', ')', '', '', '<eos>']
translated prediction
 def <unk> ( string ) : 
 	 return list ( string )  


In [136]:
# Evaluating the Generated Python code

In [134]:
# replace the unk token with a name - test
for_evaluation = processed_prediction.replace('<unk>', 'test')
exec(for_evaluation)

In [135]:
# If no error, then the run is successful

In [ ]:
# Attention

In [ ]:
def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()